In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import tensorflow as tf
import numpy as np

In [4]:
featurizer = malaya_speech.tf_featurization.STTFeaturizer()

In [5]:
x = tf.placeholder(tf.float32, (None, None))
x_len = tf.placeholder(tf.int32, (None))

In [6]:
batch_size = tf.shape(x)[0]
features = tf.TensorArray(dtype = tf.float32, size = batch_size, dynamic_size = True, infer_shape = False)
features_len = tf.TensorArray(dtype = tf.int32, size = batch_size)

init_state = (0, features, features_len)

def condition(i, features, features_len):
    return i < batch_size

def body(i, features, features_len):
    f = featurizer(x[i, :x_len[i]])
    f_len = tf.shape(f)[0]
    return i + 1, features.write(i, f), features_len.write(i, f_len)

_, features, features_len = tf.while_loop(condition, body, init_state)
features_len = features_len.stack()
padded_features = tf.TensorArray(dtype = tf.float32, size = batch_size)
maxlen = tf.reduce_max(features_len)

init_state = (0, padded_features)

def condition(i, padded_features):
    return i < batch_size

def body(i, padded_features):
    f = features.read(i)
    f = tf.pad(f, [[0, maxlen - tf.shape(f)[0]], [0,0]])
    return i + 1, padded_features.write(i, f)

_, padded_features = tf.while_loop(condition, body, init_state)
padded_features = padded_features.stack()
padded_features.set_shape((None, None, featurizer.num_feature_bins))

In [27]:
padded_features

<tf.Tensor 'TensorArrayStack_1/TensorArrayGatherV3:0' shape=(?, ?, 80) dtype=float32>

In [28]:
sess = tf.InteractiveSession()

In [29]:
padded, lens = malaya_speech.padding.sequence_1d([np.random.normal(size = (1000)),
                                  np.random.normal(size = (2000)),
                                  np.random.normal(size = (10000))],
                                 return_len = True)

In [30]:
sess.run(padded_features, feed_dict = {x: padded, x_len: lens}).shape

(3, 61, 80)